# LeNet model experiment
- In this notebook, I experiment that the Neural Network model which is exactly same as the LeNet performs well as a traffic sign classifier.
- And I confirmed that the LeNet model works well on the traffic sign images.

## Load the data

In [21]:
import pickle
import time

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.contrib.layers import flatten

In [3]:
train_file = './data/train.p'
test_file = './data/test.p'

with open(train_file, 'rb') as f:
    train = pickle.load(f)
with open(test_file, 'rb') as f:
    test = pickle.load(f)

In [12]:
X_train = train['features']
y_train = train['labels']
X_test = test['features']
y_test = test['labels']
print('X_train_origin.shape = ', X_train.shape)
print('y_train_origin.shape = ', y_train.shape)
print('X_test.shape = ', X_test.shape)
print('y_test.shape = ', y_test.shape)

X_train_origin.shape =  (39209, 32, 32, 3)
y_train_origin.shape =  (39209,)
X_test.shape =  (12630, 32, 32, 3)
y_test.shape =  (12630,)


## Preprocess the data

In [13]:
# Normalize the data

if np.max(X_train) > 1:
    X_train = X_train / 255
else:
    print('Train data is already normalized.')
    
if np.max(X_test) > 1:
    X_test = X_test / 255
else:
    print('Test data is alreadly normalized.')

In [14]:
# Split the data
from sklearn.model_selection import train_test_split

X_train_sub, X_validation, y_train_sub, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
print('Number of the original train data: ', X_train.shape[0])
print('Number of the train data for validation: ', X_train_sub.shape[0])

Number of the original train data:  39209
Number of the train data for validation:  31367


## Architecture: LeNet

In [18]:
# Hyperparameter of the model
EPOCHS = 10
BATCH_SIZE = 128

mu = 0
sigma = 0.1


# LeNet
def LeNet(x):
    # Layer 1: Convolutional.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b
    # Activation.
    conv1 = tf.nn.relu(conv1)
    # Pooling.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Layer 2: Convolutional. 
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    # Activation.
    conv2 = tf.nn.relu(conv2)
    # Pooling. 
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    # Activation.
    fc1    = tf.nn.relu(fc1)

    # Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b    
    # Activation.
    fc2    = tf.nn.relu(fc2)

    # zLayer 5: Fully Connected. Input = 84. Output = 10.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 43), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

## Train the model

In [19]:
rate = 0.001

x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=rate)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits, axis=1), tf.argmax(one_hot_y, axis=1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()


def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_train[offset: offset + BATCH_SIZE], y_train[offset: offset + BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
        
    return total_accuracy / num_examples

In [22]:
# Training the Network
from sklearn.utils import shuffle


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    t0 = time.time()
    
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print('Time: ', time.time() - t0)
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './models/LeNet/LeNet_model')
    print("Model saved")

Training...

EPOCH 1 ...
Time:  36.73797392845154
Validation Accuracy = 0.820

EPOCH 2 ...
Time:  70.90340995788574
Validation Accuracy = 0.901

EPOCH 3 ...
Time:  106.98047304153442
Validation Accuracy = 0.952

EPOCH 4 ...
Time:  142.07992601394653
Validation Accuracy = 0.968

EPOCH 5 ...
Time:  175.42979907989502
Validation Accuracy = 0.974

EPOCH 6 ...
Time:  209.98941111564636
Validation Accuracy = 0.982

EPOCH 7 ...
Time:  243.1214051246643
Validation Accuracy = 0.993

EPOCH 8 ...
Time:  278.1022069454193
Validation Accuracy = 0.993

EPOCH 9 ...
Time:  315.5771870613098
Validation Accuracy = 0.998

EPOCH 10 ...
Time:  350.7036979198456
Validation Accuracy = 0.992

Model saved


## Test the model

In [23]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('./models/LeNet/'))
    
    test_accuracy = evaluate(X_test, y_test)
    print('Test accuracy = {:.3f}'.format(test_accuracy))

Test accuracy = 0.983
